In [ ]:
import json
import time
import threading

import traitlets
import ipywidgets as ipw

from aiidalab_widgets_base import WizardAppWidget
from aiidalab_widgets_base import WizardAppWidgetStep


class ConfigureOrderStep(ipw.HBox, WizardAppWidgetStep):

    disabled = traitlets.Bool()
    configuration = traitlets.Dict(allow_none=True)

    def __init__(self, **kwargs):
        # Setup widgets for the pizza configuration
        self.style = ipw.RadioButtons(
            options=["Neapolitan", "Chicago", "New York-Style", "Detroid"],
            description="Style:",
            value=None,
        )
        self.style.observe(self._update_state, ["value"])

        self.toppings = ipw.SelectMultiple(
            options=["pepperoni", "pineapple", "anchovies"],
            description="Toppings:",
        )
        self.toppings.observe(self._update_state, ["value"])

        # Clicking on the "Confirm configuration" button locks the
        # current configuration and enables the "order" step.
        # The pizza configuration is exposed as the "configuration" trait.
        self.confirm_button = ipw.Button(
            description="Confirm configuration", disabled=True
        )
        self.confirm_button.on_click(self._confirm_configuration)

        # We need to update the step's state whenever the configuration is changed.
        self.observe(self._update_state, ["configuration"])

        super().__init__([self.style, self.toppings, self.confirm_button], **kwargs)

    def _confirm_configuration(self, button):
        "Confirm the pizza configuration and expose as trait."
        button.disabled = True
        self.configuration = dict(style=self.style.value, toppings=self.toppings.value)
        
    def reset(self):
        with self.hold_trait_notifications():
            self.style.value = None
            self.toppings.value = []
            self.configuration = {}

    def _update_state(self, _=None):
        """Update the step's state based on traits and widget state.

        The step state influences the representation of the step (e.g. the "icon") and
        whether the "Next step" button is enabled.
        """

        if self.configuration:
            # The configuration is non-empty, we can move on to the next step.
            self.state = WizardAppWidget.State.SUCCESS
        elif self.style.value and self.toppings.value:
            # Both style and topping selection has been made, the step is considered
            # to be in the "configured" state. This enables the "Confirm configuration"
            # button.
            self.state = WizardAppWidget.State.CONFIGURED
        elif self.style.value or self.toppings.value:
            self.state = WizardAppWidget.State.READY
        else:
            # In all other cases the step is always considered to be in the "init" state.
            self.state = WizardAppWidget.State.INIT

    @traitlets.observe("state")
    def _observe_state(self, change):
        # Enable the confirm button in case that the pizza has been configured.
        with self.hold_trait_notifications():
            self.disabled = change["new"] == WizardAppWidget.State.SUCCESS
            self.confirm_button.disabled = change["new"] is not WizardAppWidget.State.CONFIGURED
        
    @traitlets.observe("disabled")
    def _observe_disabled(self, change):
        with self.hold_trait_notifications():
            for child in self.children:
                child.disabled = change["new"]


class ReviewAndSubmitOrderStep(ipw.VBox, WizardAppWidgetStep):

    # We use traitlets to connect the different steps.
    # Note that we can use dlinked transformations, they do not need to be of the same type.
    configuration = traitlets.Dict()

    # We will keep track of the order status with this Enum trait.
    order_status = traitlets.Enum(
        ("", "ordered", "in-transit", "delivered", "unavailable")
    )

    def __init__(self, **kwargs):
        # The pizza configuration is represented as a formatted dictionary.
        self.configuration_label = ipw.HTML()

        # The second step has only function, executing the order by clicking on this button.
        self.order_button = ipw.Button(description="Submit order", disabled=True)
        self.order_button.on_click(self.submit_order)

        # We update the step's state whenever there is a change to the configuration or the order status.
        self.observe(self._update_state, ["configuration", "order_status"])

        super().__init__([self.configuration_label, self.order_button], **kwargs)
        
    def reset(self):
        self.order_status = ""

    @traitlets.default("order_status")
    def _default_order_status(self):
        # Need to explicitly initialize the order status.
        return ""

    @traitlets.observe("configuration")
    def _observe_configuration(self, change):
        "Format and show the pizza configuration."
        if change["new"]:
            self.configuration_label.value = f"<h4>Configuration</h4><pre>{json.dumps(change['new'], indent=2)}</pre>"
        else:
            self.configuration_label.value = (
                "<h4>Configuration</h4>[Please configure your pizza]"
            )

    def submit_order(self, button):
        "Submit the order and simulate the delivery."
        button.disabled = True
        self.order_status = "ordered"

        def process_order():
            time.sleep(5)

            # We simulate a failure by pretending that "anchovies" are out-of-stock.
            if "anchovies" in self.configuration.get("toppings", []):
                self.order_status = "unavailable"
                return

            self.order_status = "in-transit"
            time.sleep(3)
            self.order_status = "delivered"

        threading.Thread(target=process_order).start()

    def _update_state(self, _=None):
        "Update the step's state based on the order status and configuration traits."
        if self.order_status:  # the order has been submitted
            self.state = WizardAppWidget.State.SUCCESS
        elif self.configuration:  # the order can be submitted
            self.state = WizardAppWidget.State.CONFIGURED
        else:
            self.state = WizardAppWidget.State.INIT

    @traitlets.observe("state")
    def _observe_state(self, change):
        """Enable the order button once the step is in the "configured" state."""
        self.order_button.disabled = change["new"] != WizardAppWidget.State.CONFIGURED


class TrackOrderStep(ipw.VBox, WizardAppWidgetStep):

    # We retrieve the order status from the previous step, but instead of an Enum
    # we just use a string type. In this way we are a bit more forgiving about showing
    # "unexpected" messages such as error conditions.
    order_status = traitlets.Unicode()

    def __init__(self, **kwargs):
        # The order status is shown as a text label.
        self.status_label = ipw.Text(description="Order status", disabled=True)
        ipw.dlink((self, "order_status"), (self.status_label, "value"))

        # We update the step's state whenever there is a change to the configuration or the order status.
        self.observe(self._update_state, ["order_status"])

        super().__init__([self.status_label], **kwargs)
        
    def can_reset(self):
        return self.state is not WizardAppWidget.State.ACTIVE

    def _update_state(self, _=None):
        "Update the step's state based on the order status configuration traits."
        if self.order_status in ("ordered", "in-transit"):
            self.state = WizardAppWidget.State.ACTIVE
        elif self.order_status is "delivered":
            self.state = WizardAppWidget.State.SUCCESS
        elif self.order_status:
            self.state = WizardAppWidget.State.FAIL
        else:
            self.state = WizardAppWidget.State.INIT


# Setup all steps of the app. Setting the `auto_next` trait to True makes it
# so that the next step is automatically selected once the previous step is
# in the "success" state.
configure_order_step = ConfigureOrderStep(auto_next=True)
review_and_submit_order_step = ReviewAndSubmitOrderStep(auto_next=True)
track_order_step = TrackOrderStep()


# Data that is communicated from one step to the next via traits.
ipw.dlink(
    (configure_order_step, "configuration"),
    (review_and_submit_order_step, "configuration"),
)
ipw.dlink(
    (review_and_submit_order_step, "order_status"), (track_order_step, "order_status")
)

# Setup the app by adding the various steps in order.
app = WizardAppWidget(
    steps=[
        ("Configure pizza", configure_order_step),
        ("Order pizza", review_and_submit_order_step),
        ("Track order", track_order_step),
    ]
)


# Display the app to the user.
display(app)